In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import tensorflow as tf
import numpy as np
import re

In [42]:
filename = "/content/drive/My Drive/Alice2.txt"
raw_text = open(filename).read()
print(raw_text[:100])

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on t


In [0]:
# create mapping of unique words to integers, and reverse
raw_text_words=re.findall(r"[\w]+|[.!?;]", raw_text)
#raw_text_words = raw_text.split()
#print(raw_text_words)

In [0]:
words = sorted(list(set(raw_text_words)))
#word to integer mapping
word_to_int = dict((w, i) for i, w in enumerate(words))
#integer to word mapping
int_to_word = dict((i, w) for i, w in enumerate(words))

In [0]:
#print(word_to_int)
#print(int_to_word)
#print(words)

In [46]:
n_words = len(raw_text_words)
n_vocab = len(words)
print("Total Words in the document: ", n_words)
print("Total Vocab: ", n_vocab)

Total Words in the document:  4614
Total Vocab:  1013


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
    seq_in = raw_text_words[i:i + seq_length]
    seq_out = raw_text_words[i + seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])

In [48]:
dataX[:5]

[[18,
  55,
  1,
  30,
  883,
  101,
  52,
  9,
  961,
  204,
  910,
  430,
  948,
  909,
  644,
  806,
  232,
  475,
  803,
  648,
  883,
  190,
  169,
  644,
  466,
  636,
  910,
  324,
  649,
  656],
 [55,
  1,
  30,
  883,
  101,
  52,
  9,
  961,
  204,
  910,
  430,
  948,
  909,
  644,
  806,
  232,
  475,
  803,
  648,
  883,
  190,
  169,
  644,
  466,
  636,
  910,
  324,
  649,
  656,
  932],
 [1,
  30,
  883,
  101,
  52,
  9,
  961,
  204,
  910,
  430,
  948,
  909,
  644,
  806,
  232,
  475,
  803,
  648,
  883,
  190,
  169,
  644,
  466,
  636,
  910,
  324,
  649,
  656,
  932,
  782],
 [30,
  883,
  101,
  52,
  9,
  961,
  204,
  910,
  430,
  948,
  909,
  644,
  806,
  232,
  475,
  803,
  648,
  883,
  190,
  169,
  644,
  466,
  636,
  910,
  324,
  649,
  656,
  932,
  782,
  449],
 [883,
  101,
  52,
  9,
  961,
  204,
  910,
  430,
  948,
  909,
  644,
  806,
  232,
  475,
  803,
  648,
  883,
  190,
  169,
  644,
  466,
  636,
  910,
  324,
  649,
  656,
  

In [49]:
dataY[:5]

[932, 782, 449, 676, 511]

In [50]:
int_to_word[9]

'Alice'

In [51]:
int_to_word[30]

'Down'

In [52]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  4584


In [0]:
# reshape X to be [samples, time steps, features]
x_train = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
x_train = x_train / float(n_vocab)
# one hot encode the output variable
y_train = np.eye(n_vocab)[dataY]

In [54]:
print(x_train[:2])

[[[0.017769  ]
  [0.05429418]
  [0.00098717]
  [0.029615  ]
  [0.87166831]
  [0.09970385]
  [0.05133268]
  [0.0088845 ]
  [0.94866732]
  [0.20138203]
  [0.89832182]
  [0.42448174]
  [0.93583416]
  [0.89733465]
  [0.63573544]
  [0.79565647]
  [0.2290227 ]
  [0.46890424]
  [0.79269497]
  [0.63968411]
  [0.87166831]
  [0.1875617 ]
  [0.16683119]
  [0.63573544]
  [0.46001974]
  [0.6278381 ]
  [0.89832182]
  [0.31984205]
  [0.64067127]
  [0.64758144]]

 [[0.05429418]
  [0.00098717]
  [0.029615  ]
  [0.87166831]
  [0.09970385]
  [0.05133268]
  [0.0088845 ]
  [0.94866732]
  [0.20138203]
  [0.89832182]
  [0.42448174]
  [0.93583416]
  [0.89733465]
  [0.63573544]
  [0.79565647]
  [0.2290227 ]
  [0.46890424]
  [0.79269497]
  [0.63968411]
  [0.87166831]
  [0.1875617 ]
  [0.16683119]
  [0.63573544]
  [0.46001974]
  [0.6278381 ]
  [0.89832182]
  [0.31984205]
  [0.64067127]
  [0.64758144]
  [0.92003949]]]


In [55]:
print(y_train[2])

[0. 0. 0. ... 0. 0. 0.]


In [56]:
print(x_train.shape)

(4584, 30, 1)


In [57]:
print(y_train.shape)

(4584, 1013)


In [0]:
batch_size = y_train.shape[0]

In [59]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32,  shape=(None, seq_length, 1))
Y = tf.placeholder(tf.float32,  shape=(None, n_vocab))
num_layers = 2   
hidden_units=200
#rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_units, activation=tf.nn.tanh)
layers = [tf.nn.rnn_cell.BasicLSTMCell(num_units=hidden_units, activation=tf.nn.tanh)
        for _ in range(num_layers)]

cells = tf.nn.rnn_cell.MultiRNNCell(layers)
outputs, state = tf.nn.dynamic_rnn(cells,X,dtype=tf.float32)
logits = tf.layers.dense(outputs[:,-1], n_vocab)
prob = tf.nn.softmax(logits)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                             logits=logits, labels=Y))
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
acc = tf.reduce_mean(tf.cast(
                        tf.equal(
                            tf.argmax(logits, 1),
                            tf.argmax(Y, 1),
                        ),
                        tf.float32,
                    ))


In [60]:

epochs =200
batch_size=100
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#t0,t1, t2=sess.run([outputs,logits, prob], feed_dict={X:x_train, Y:y_train})
for i in range(epochs):
  for j in range(0, x_train.shape[0],batch_size):
    x_batch = x_train[j:j+batch_size]
    y_batch = y_train[j:j+batch_size]
    loss_, train_, acc_ = sess.run([loss,train,acc], feed_dict={X:x_batch, Y:y_batch})
  print("loss:",loss_," Accuracy:",acc_)

loss: 6.149459  Accuracy: 0.071428575
loss: 5.9320927  Accuracy: 0.071428575
loss: 5.8520284  Accuracy: 0.071428575
loss: 5.809623  Accuracy: 0.071428575
loss: 5.7849255  Accuracy: 0.071428575
loss: 5.7697515  Accuracy: 0.071428575
loss: 5.759843  Accuracy: 0.071428575
loss: 5.752988  Accuracy: 0.071428575
loss: 5.747976  Accuracy: 0.071428575
loss: 5.7441177  Accuracy: 0.071428575
loss: 5.741017  Accuracy: 0.071428575
loss: 5.738439  Accuracy: 0.071428575
loss: 5.736239  Accuracy: 0.071428575
loss: 5.7343225  Accuracy: 0.071428575
loss: 5.7327642  Accuracy: 0.071428575
loss: 5.7311244  Accuracy: 0.071428575
loss: 5.729799  Accuracy: 0.071428575
loss: 5.7285476  Accuracy: 0.071428575
loss: 5.7275352  Accuracy: 0.071428575
loss: 5.726568  Accuracy: 0.071428575
loss: 5.7256265  Accuracy: 0.071428575
loss: 5.7244925  Accuracy: 0.071428575
loss: 5.723484  Accuracy: 0.071428575
loss: 5.7225857  Accuracy: 0.071428575
loss: 5.7216  Accuracy: 0.071428575
loss: 5.7205634  Accuracy: 0.071428575


In [61]:
print(outputs)

Tensor("rnn/transpose_1:0", shape=(?, 30, 200), dtype=float32)


In [0]:
def encode_test(txt):
  raw_text_words=re.findall(r"[\w]+|[.!?;-]", txt)
  if len(raw_text_words) != seq_length:
     raw_text_words = ["!"]*(seq_length-len(raw_text_words))+raw_text_words
  return [word_to_int[word] for word in raw_text_words]
  

In [0]:
# pick a random seed
# start = np.random.randint(0, len(dataX)-1)
# #start=0
# pattern = dataX[start]

In [0]:
#print(start)

In [69]:
test="The rabbit was sitting by"
pattern = encode_test(test)
print(pattern)
print(len(pattern))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 118, 718, 961, 806, 232]
30


In [70]:
print("Seed:")
print (' '.join([int_to_word[value] for value in pattern]))

Seed:
! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! The rabbit was sitting by


In [0]:
# generate text
out=" "
for i in range(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = sess.run(prob,feed_dict={X:x})
    #print(prob.shape)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    out=out+ result+ " "
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [72]:
print(out)

 she ! she ! mouse ! either the to and the the ! she to know . Let me ! m opening it the the presents that poor for I ! Antipathies I think she was like glad in . However was forgot ! cried says Mouse m station other and she ! I I then are in knowledge of history very night Alice to bring tears again . Dinah Alice THAT S the great puzzle to to and the on table the it was a the a it the too and but she it I round when was on the she going put into into the garden and and but to had upon a little three legged table the poor she the table table she was was size had stockings she she a it when too slippery but and when little hippopotamus but then she remembered how small she she she the table she she a and she go ! cried I to look I cried Alice hastily afraid that she had hurt the poor animal s feelings . I quite forgot you didn t like . ! Not ! ! ESQ . HEARTHRUG NEAR THE FENDER WITH ALICE 
